# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import confinement_mode_classifier as cmc
import torchvision
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import re

path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Seed set to 42


Device: cuda:0


In [2]:

data_dir_path = f'{path}/data/LH_alpha'
file_names = os.listdir(data_dir_path)


#Chose what shots will be used in notebook. Removed shots has different dimensions
batch_size = 32
shot_numbers = [re.search(r'shot_(\d+)', file_name).group(1) for file_name in file_names]
removed_shots = ['13182', '20009','20112', '20143', '20145', '20146', '20147', '16989']
shot_numbers = [valid_shot for valid_shot in shot_numbers if valid_shot not in removed_shots]

shots_for_testing = ['18130', '16773', '16534', '19094', '18133']
shots_for_validation = ['16769', '19379', '18057', '18132']

shot_df, test_df, _, _ = cmc.load_and_split_dataframes(path, shot_numbers, shots_for_testing, shots_for_validation=shots_for_validation)


### Load the model

In [3]:
#Initiate a single camera model
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

#Which model to test?
model_path = f'{path}/runs/15-01-2024, 16-29-59 RIS1xRIS2_all_layers'


#TEST one camera model
# pretrained_model.load_state_dict(torch.load(f'{model_path}/model_fc_trained.pt'))
# pretrained_model.eval()
# pretrained_model.to(device)
# test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, balance_data=False)


#TEST ensembled model
ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
ensembled_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
ensembled_model.eval()
ensembled_model.to(device)

test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, 
                                  balance_data=False, second_img_opt='RIS2', ris_option='RIS1')

### Test model on unbalanced test dset

In [ ]:
metrics = cmc.test_model(model_path, ensembled_model, test_dataloader)

### Here the model is tested on individual shots (generates time-confidence graph)

In [ ]:
shots_for_testing = list(map(int,shots_for_testing))
img_path = cmc.per_shot_test(path=model_path, shots=shots_for_testing, results_df=metrics['prediction_df'])

Seed set to 42


Device: cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]